In [18]:
from astropop.astrometry import AstrometrySolver, solve_astrometry_framedata, solve_astrometry_xy
from astropop.photometry import background, starfind
from astropop.framedata import FrameData, read_framedata
from astropy.io import fits
from astropy.coordinates import Angle

from astroquery.skyview import SkyView

In [19]:
image_name = '/home/julio/Astrometry/M67.fits'
image_ra = '08:50:00'
image_dec = '11:50:00'
search_radius = '1 deg'
indexes_path = '/home/julio/Astrometry/INDEXES/5200'

# download DSS image of M67 using SkyView
dss_image = SkyView.get_images(position='M67',
                               survey='DSS',
                               radius=Angle('0.2 deg'))[0][0]
dss_image.writeto(image_name, overwrite=True)

# AstrometrySolver class

In [20]:
a = AstrometrySolver()

In [21]:
solution = a.solve_field(image_name,
                         options={'add_path': indexes_path,
                                  'ra': image_ra, 'dec': image_dec, 'radius': search_radius},
                         stdout_loglevel='INFO')

2023-06-13 19:25:14,506 astropop - INFO - Runing: /home/julio/mambaforge/envs/astropop/bin/solve-field /home/julio/Astrometry/M67.fits --dir /tmp/M67_i_1iwddz_astrometry.net --config /tmp/M67_i_1iwddz_astrometry.net/M67.cfg --ra 132.5 --dec 11.833333333333334 --radius 1.0 --no-plots --overwrite --corr /tmp/M67_i_1iwddz_astrometry.net/M67.corr  [py_utils]
2023-06-13 19:25:14,519 astropop - INFO - Reading input file 1 of 1: "/home/julio/Astrometry/M67.fits"...  [py_utils]
2023-06-13 19:25:14,598 astropop - INFO - Found an existing WCS header, will try to verify it.  [py_utils]
2023-06-13 19:25:14,598 astropop - INFO - Extracting sources...  [py_utils]
2023-06-13 19:25:14,701 astropop - INFO - simplexy: found 305 sources.  [py_utils]
2023-06-13 19:25:15,889 astropop - INFO - Solving...  [py_utils]
2023-06-13 19:25:16,217 astropop - INFO - Reading file "/tmp/M67_i_1iwddz_astrometry.net/M67.axy"...  [py_utils]
2023-06-13 19:25:16,218 astropop - INFO - Only searching for solutions within 1 d

In [24]:
print(solution.wcs)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 132.845996628  11.8137489136  
CRPIX : 150.5  150.5  
CD1_1 CD1_2  : -0.000666646299846  1.75696203662e-07  
CD2_1 CD2_2  : 1.17094852931e-07  0.000666605100714  
NAXIS : 0  0


# solve_astrometry_xy

In [25]:
frame = read_framedata(image_name)
bkg, rms = background(frame.data)
sources = starfind(frame.data, background=bkg, noise=rms, threshold=20)
width, height = frame.shape

KeyError: "Keyword 'HISTORY' not found."

In [ ]:
sol = solve_astrometry_xy(sources['x'], sources['y'], sources['flux'],
                          width=width, height=height,
                          options={'add_path': indexes_path,
                                  'ra': image_ra, 'dec': image_dec, 'radius': search_radius,
                                  'crpix-center': None, 'tweak-order': 4},
                          stdout_loglevel='INFO')

In [ ]:
sol.wcs

# solve_astrometry_framedata

In [ ]:
sol = solve_astrometry_framedata(frame,
                                 options={'add_path': indexes_path,
                                          'ra': image_ra, 'dec': image_dec, 'radius': search_radius,
                                          'crpix-center': None, 'tweak-order': 4},
                                 stdout_loglevel='INFO')

In [ ]:
sol.wcs

# AstrometricSolution

In [26]:
solution.wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 132.845996628  11.8137489136  
CRPIX : 150.5  150.5  
CD1_1 CD1_2  : -0.000666646299846  1.75696203662e-07  
CD2_1 CD2_2  : 1.17094852931e-07  0.000666605100714  
NAXIS : 0  0

In [27]:
solution.header

SIMPLE  =                    T / Standard FITS file                             
BITPIX  =                    8 / ASCII or bytes array                           
NAXIS   =                    0 / Minimal header                                 
EXTEND  =                    T / There may be FITS ext                          
WCSAXES =                    2 / no comment                                     
CTYPE1  = 'RA---TAN-SIP' / TAN (gnomic) projection + SIP distortions            
CTYPE2  = 'DEC--TAN-SIP' / TAN (gnomic) projection + SIP distortions            
EQUINOX =               2000.0 / Equatorial coordinates definition (yr)         
LONPOLE =                180.0 / no comment                                     
LATPOLE =                  0.0 / no comment                                     
CRVAL1  =        132.845996628 / RA  of reference point                         
CRVAL2  =        11.8137489136 / DEC of reference point                         
CRPIX1  =                150

In [28]:
solution.correspondences

field_x,field_y,field_ra,field_dec,index_x,index_y,index_ra,index_dec,index_id,field_id,match_weight,FLUX,BACKGROUND
pixels,pixels,degrees,degrees,pixels,pixels,degrees,degrees,none,none,none,unknown,unknown
float64,float64,float64,float64,float64,float64,float64,float64,int32,int32,float64,float32,float32
34.85548400878906,20.532312393188477,132.92485923212246,11.726919008467611,35.05859699792801,20.642018010728464,132.9247207495702,11.72699240886807,11,0,0.9898638113267965,9302.139,3911.8613
46.37539291381836,248.11927795410156,132.91698566957803,11.878738007165833,46.44105277142182,248.07053263630593,132.91694086840715,11.87870558548878,101,1,0.9901592131420947,9018.018,4202.9824
158.05941772460938,12.409916877746582,132.84087281675698,11.721595802636996,157.95063650953037,12.399268873090847,132.84094685781446,11.721588701279531,92,2,0.9901705766448283,9122.119,3944.8809
296.3533020019531,85.46388244628906,132.74673394046354,11.77032696042933,296.32845369070515,85.37248838710052,132.746750859307,11.770265999806785,41,5,0.9901176377427999,9003.846,3964.1543
297.58489990234375,154.1095428466797,132.7459130066678,11.816085710101218,297.5441019357298,153.9454298381467,132.7459407077159,11.815976361442809,77,6,0.9900533600010665,8981.352,4003.6484
104.5924072265625,160.32260131835938,132.87727315205524,11.820283507708053,104.47654764184021,160.39149675577605,132.8773520589947,11.820329419233394,91,7,0.9902863961611872,8815.066,4179.9336
17.85403823852539,55.490848541259766,132.93643765005882,11.750252499056753,17.888205105803195,55.637655475439246,132.93641435158702,11.750350535263108,76,10,0.9900425018276018,9073.695,3964.3047
271.6463928222656,74.68538665771484,132.76353614430153,11.763156055690384,271.6696423810698,74.6869161967365,132.76352036461677,11.763157050016662,17,12,0.9902069422150418,8988.049,3990.9512
